# Model Notebook

this is the notebook for model in age and gender detector.here,we will extract the data,train it and fir it to our model and evaluate our results and optimize the model for better results.

In [ ]:
import tensorflow as tf 
import cv2 
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense,MaxPool2D,Conv2D,MaxPooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Activation,Add
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam,Adagrad,Adadelta,Adamax,RMSprop
from keras.activations import relu, elu, linear, sigmoid

### Extracting data from dataset

#### Plotting the samples agewise

In [ ]:
fldr ="C:/Users/Aman/New folder/UTKFace"

#### Create list of images of dataset

In [ ]:
import os
files = os.listdir(fldr)

In [ ]:
ages=[]
genders=[]
images=[]

for fle in files:
    age=int(fle.split("_")[0])
    gender=int(fle.split("_")[1])
    total=fldr+"/"+fle
    print(total)
    image=cv2.imread(total)

    image=cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image=cv2.resize(image,(48,48))
    images.append(image)

#### Create list of age and gender 

In [ ]:
for fle in files:
    age=int(fle.split("_")[0])
    gender=int(fle.split("_")[1])
    ages.append(age)
    genders.append(gender)

In [ ]:
plt.imshow(images[0])

In [ ]:
print(ages[0])

In [ ]:
print(genders[0])

In [ ]:
images_f=np.array(images)
ages_f=np.array(ages)
genders_f=np.array(genders)

In [ ]:
np.save(fldr+"images.npy",images_f)
np.save(fldr+"ages.npy",ages_f)
np.save(fldr+"genders.npy",genders_f)

#### Finding the number of male and female samples

In [ ]:
values,counts=np.unique(genders,return_counts=True)
print(counts)

#### Potting the numbers of male and female samples

In [ ]:
fig=plt.figure()
ax=fig.add_axes([0,0,1,1])
gender=["Male","Female"]
values=[12391,11317]
ax.bar(gender,values)
plt.show()

#### Finding the number of sample in each age

In [ ]:
values,counts=np.unique(ages_f,return_counts=True)
print(counts)

#### Converting age counts to list 

In [ ]:
val=values.tolist()
cnt=counts.tolist()


#### Plotting the samples agewise

In [ ]:
plt.plot(counts)
plt.title("Distribution of ages")
plt.show()

In [ ]:
labels=[]
i=0
while i<len(ages):
    label=[]
    label.append(ages[i])
    label.append(genders[i])
    labels.append(label)
    i=i+1

In [ ]:
images_f_2=images_f/255

In [ ]:
labels_f=np.array(labels)

In [ ]:
images_f_2.shape

 #### Splitting dataset into train and test

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(images_f_2,labels_f,test_size=0.25)

In [ ]:
y_train[0:5]

#### Splitting y_train and y_test further for better understanding

In [ ]:
y_train_2=[y_train[:,1],y_train[:,0]]
y_test_2=[y_test[:,1],y_test[:,0]]

In [ ]:
y_train_2[0][0:5]

In [ ]:
y_train_2[1][0:5]

#### Defining the model

In [ ]:
def Convolution(input_tensor,filters):
    x=Conv2D(filters=filters,kernel_size=(3,3),padding="same",strides=(1,1),kernel_regularizer=l2(0.001))(input_tensor)
    x=Dropout(0.1)(x)
    x=Activation("relu")(x)
    return x

In [ ]:
def model(input_shape):
    inputs=Input((input_shape))
    conv_1=Convolution(inputs,32)
    maxp_1=MaxPooling2D(pool_size=(2,2))(conv_1)
    conv_2=Convolution(maxp_1,64)
    maxp_2=MaxPooling2D(pool_size=(2,2))(conv_2)
    conv_3=Convolution(maxp_2,128)
    maxp_3=MaxPooling2D(pool_size=(2,2))(conv_3)
    conv_4=Convolution(maxp_3,256)
    maxp_4=MaxPooling2D(pool_size=(2,2))(conv_4)
    flatten=Flatten()(maxp_4)
    dense_1=Dense(64,activation="relu")(flatten)
    dense_2=Dense(64,activation="relu")(flatten)
    drop_1=Dropout(0.2)(dense_1)
    drop_2=Dropout(0.2)(dense_2)
    output_1=Dense(1,activation='sigmoid',name="gender_out")(drop_1)
    output_2=Dense(1,activation="relu",name="age_out")(drop_2)
    model=Model(inputs=[inputs],outputs=[output_1,output_2])
    model.compile(loss=["binary_crossentropy","mae"],optimizer="Adam",metrics=[["accuracy"],["accuracy"]])
    return model


In [ ]:
Model = model((48,48,3))

In [ ]:
Model.summary()

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [ ]:
fle_s="Age_Gender_Detector.weights.keras"
cheakpoint=ModelCheckpoint(fle_s,monitor="val_loss",verbose=1,save_best_only=True,save_weights_only=False,mode="auto",save_freq="epoch")
Early_stop=tf.keras.callbacks.EarlyStopping(patience=75,monitor="val_loss",restore_best_weights=True)
callback_list=[cheakpoint,Early_stop]


In [ ]:
History=Model.fit(x_train,y_train_2,batch_size=64,validation_data=(x_test,y_test_2),epochs=250,callbacks=callback_list)

#### Model Evaluation

In [ ]:
Model.evaluate(x_test,y_test_2)

In [ ]:
pred=Model.predict(x_test)

In [ ]:
pred[1]

#### Plotting Loss

In [ ]:
plt.plot(History.history["loss"])
plt.plot(History.history["val_loss"])
plt.title("Model loss")
plt.xlabel=("Epoch")
plt.ylabel=("Loss")
plt.legend(["Train","Validation"],loc="upper left")
plt.subplots_adjust(top=1.0,bottom=0.0,right=0.95,left=0,hspace=0.25,wspace=0.35)

#### Plotting Gender Accuracy

In [ ]:
plt.plot(History.history["gender_out_accuracy"])
plt.plot(History.history["val_gender_out_accuracy"])
plt.title("Model accuracy")
plt.legend(["Train","Validation"],loc="upper left")
plt.subplots_adjust(top=1.0,bottom=0.0,right=0.95,left=0,hspace=0.25,wspace=0.35)


#### Plotting Pridicted and Actual age

In [ ]:
fig,ax=plt.subplots()
ax.scatter(y_test_2[1],pred[1])
ax.plot([y_test_2[1].min(),y_test_2[1].max()],[y_test_2[1].min(),y_test_2[1].max()],"k--",lw=4)
ax.set_xlabel("Actual age")
ax.set_ylabel("Predicted age")
plt.show()

#### Making Report of the Model(Gender Only)

In [ ]:
i=0
pred_l=[]
while(i<len(pred[0])):
    pred_l.append(int(np.round(pred[0][i])))
    i=i+1

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
report=classification_report(y_test_2[0],pred_l)

In [ ]:
print(report)

In [ ]:
result=confusion_matrix(y_test_2[0],pred_l)

In [ ]:
import seaborn as sns

In [ ]:
sns.heatmap(result,annot=True,cmap="Spectral")

#### Testing our Images Ourselves

In [ ]:
def test_image(ind,images_f,images_f_2,Model):
    plt.imshow(images_f[ind])
    image_test=images_f_2[ind]
    pred_l=Model.predict(np.array([image_test]))
    gender_f=["Male","Female"]
    age=int(np.round(pred_l[1][0]))
    gender=int(np.round(pred_l[0][0]))
    print("Predicted age : "+ str(age))
    print("Predicted gender : "+ gender_f[gender])

In [ ]:
test_image(4,images_f,images_f_2,Model)

In [ ]:
test_image(23,images_f,images_f_2,Model)

In [ ]:
test_image(168,images_f,images_f,Model)

In [ ]:
test_image(888,images_f,images_f_2,Model)

In [ ]:
test_image(1288,images_f,images_f_2,Model)